# Determining Level Flight Velocity

Following Walter Erbach's lead, let's try to find the forward velocity needed for the Wart to maintain level flight.

First, we need to load the basic model airframe data and the airfoil coefficient data:

In [25]:
import mmtime_helper
from mmtime.utils import (
    get_points,
    fit_curve,
    show_curve,
)
import numpy as np

## Coordinate atransformations

so far, we have specified locations in a reference coordinate system whose origin is at the nose of the motor stick. In order to calculate the moments about the center of gravity, it will be convenient to express locations in a system aligned with the CG and rotated so the **X** axis is aligned with the motor stick at some angle of attack.

We could dig out our trig books and figure out the arms, or we can just perform coordinate transformations numerically. With a little bit of matrix math, this will be fairly simple:

Here are the definitions of the transformation matrices we need:

Here is the translation matrix:

\begin{equation}
T = 
\begin{pmatrix}
1 & 0 & -dx \\
0 & 1 & -dy \\
0 & 0 & 1
\end{pmatrix}
\end{equation}

And here is the rotation matrix:

\begin{equation}
R = 
\begin{pmatrix}
\cos\alpha & \sin\alpha & 0 \\
-\sin\alpha & \cos\alpha & 0 \\
0 & 0 & 1
\end{pmatrix}
\end{equation}

We transform our coordinates using this equation:
\begin{equation}
X' = R T X
\end{equation}

where $X$ is a simple matrix:

\begin{equation}\label{eq:}
X = 
\begin{pmatrix}
x \\
y
\\1
\end{pmatrix}
\end{equation}

**numpy** will do this transformation for us:


Here are two functions that create transformation matrices we will need: 

In [26]:
def rotation(angle):
    angle = np.radians(angle)
    return np.array([
        [np.cos(angle), np.sin(angle), 0],
        [-np.sin(angle),  np.cos(angle), 0],
        [0, 0, 1]
    ])

def translation(tx, ty):
    return np.array([
        [1, 0, -tx],
        [0, 1, -ty],
        [0, 0, 1]
    ])

Here is an example of coordinst translation. We are moving the origin ot (3,3) and finding the original origin in the new system:

In [27]:
x1 = 0
y1 = 0
T = translation(3,3)
x = np.array([[x1],[y1], [1]])
a = np.matmul(T, x)
x2 = a[0][0]
y2 = a[1][0]
x2, y2

(-3, -3)

That makes sense. Not let's rotate the new system 45 degrees counter clockwise. This will align the **x** axis so it goes through the original origin:


In [28]:
R = rotation(45)
b = np.matmul(R, a)
x2 = b[0][0]
y2 = b[1][0]
x2,y2

(-4.242640687119286, -4.440892098500626e-16)

That looks right. We have a scheme to calculate the moment arms for our model at some angle of attack.

We will be calculating moments for the lift, drag and thrust vectors in this transformed coordinate system. A lift force acting at negative **x** values will of the CG will generate a nose up moment, so we will have to multiply by negative **x** when we calculate moments. The drag force  will be multiplied by positive **Y** values. 

In [29]:
## Basic Moment Calculation

At the moment, I do not have good airfoil data for the airfoils on the Wart, so I will use the 3% arc for both wing and stab. We need to generate the Cl curve fit function for this calculation. I wrapped up all the data needed for this calculation using three Python classes, one for model dimensions, one for airfoil coefficients, and one foe the standard atmosphere properties we need. These classes use code we have already seen in this study. You can see all of the code used here on the project website. Here is how we use it:

We need the air density for this calculation. The **StdAtm** module in this project provides that, using **pint** to manage units:

In [30]:
from mmtime.StdAtm import Air
import pint
u = pint.UnitRegistry()

In [31]:
density = Air(u).get_properties(864)['rho']
density

1.1265797847090362 <Unit('kilogram / meter ** 3')>

In [32]:
Cl = 1.0 # try McLean's tactic
weight = 2.06 * u.grams
wing_area = 30 * u.inch ** 2
stab_area = 15 * u.inch ** 2
v = 2 * weight / (density * Cl * (wing_area+ stab_area))
v *= u.gravity
v = v ** 0.5
v.to('mph')

2.486231765000739 <Unit('mile_per_hour')>

Gary estimated that the Wart was flying at about two miles per hour, so this calculation looks reasonable. Let's set things up to generate an angle of attach sweep with some actual model data.

## Reynolds Number

An important quantity in aerodynamic work is the Reynold's Number, a nndimensional value that relates the viscous forces to the inertial forces workingon a surface. This number is commonly used to characterize the type of airflow a vehicle might experience. For our indoor models, this number will be low meaning that the flow near the surface of our fling syrfaces should remain laminar. This means we do not need to worry about turbulence near the surface

(This section needs to be expanded)

The definition of the Reynolds NUmber is:

\begin{equation}
RE = \frac{\rho u L}{\mu}
\end{equation}

Where $\rho$ is the air density, $u$ is the flight velocity, $L$ is a reference length usually the mean chord, and $the \mu$ is the dynamic viscosity of the air. 

Here is a sample calculation:


In [33]:
mean_chord = 1.85 * u.inch
mu = Air(u).get_properties(864)['mu']
RE = density * v * mean_chord / mu
RE.to_base_units()

3338.9543875201584 <Unit('dimensionless')>

Airfoil aerodynamic coefficients vary with Reynolds Number, which presents a problem when doing an analysis. We need the aerodynamic coefficients to calculate the flight velocity, and they are tied to a Reynolds Number we will not know until we pick an airfoil. In this calculation, I simply set the lift coefficient. If I wanted to use better data, I would have to pick an airfoil based on some number I hoped would be close, do the calculations, and tune the airfoil until I got close enough to be happy with the results. 

There are ways to deal with this that I intend to explore later. Since I have a lot of background in Computational Fluid Dynamics, I am currently testing several simple open-source CFD programs that might help simplify the search for airfoil data.


## Estimated Flight Distance

One of the key assumptions in McLean's method is that the energy spend clmbing to maximum height is about equal to the energy spend descending back to the floor. This make sense if you think about potential energy. Therefore, McLean argued that we could average the flight out to a simple level fight path for the duration of the flight. We just calculated the speed, what would that predict for the distance traveled:

In [36]:
distance = v * 618 * u.seconds
distance.to_base_units()


686.873039803625 <Unit('meter')>

Moving the model that distance requires some energy:

In [37]:
KE = 0.5 * weight / u.gravity * v ** 2
KE.to_base_units()

0.0001297455704121917 <Unit('kilogram * meter')>

## Rubber Motors

According to McLean, Hacklinger, and others, the amountof energy we can get out of  our wound-up rubber motors is proportional to the weight of themotor.

\begin{equation}
E = K W_m
\end{equation}

Where, according to McLean, $K = 30,000 inches$.

The Warts motor weight 0.86 grams, so the available energy was:

In [39]:
rubber_weight = 0.86 * u.grams
K = 30000 * u.inches

E_max = K * rubber_weight
E_max.to_base_units()

0.65532 <Unit('kilogram * meter')>


This number represents a *Potential Energy* that we want to power our model. We wind up the motor, producing a torque that will be applied to the prop shaft. For the Wart, the launch torque was arounf 0.31 If we turned that much potential energy into kinetic energy, our model could be traveling at:

In [43]:
V_k = (2 * E_max/weight*u.gravity) ** 0.5
V_k.to_base_units()

78.98933120782954 <Unit('meter / second')>

Our model is certainly not moving that fast, but if we used our rubber motor as a sling-shot, we could probably throw a 2 gram chunk of lead across the room at that speed. So where did all that energy go?


In [45]:
import stl
from stl import mesh
your_mesh = mesh.Mesh.from_file('propeller.stl')
volume, cog, inertia = your_mesh.get_mass_properties()
print(inertia)

[[ 5.25428242e-01  8.91828065e-11  7.31385415e-15]
 [ 8.91828065e-11  5.24494901e-01 -2.50452413e-02]
 [ 7.31385415e-15 -2.50452413e-02  1.44970048e-02]]


The rotational energy of the propeller is the energy absorbed by spinning the mass of the propeller around the prop shaft. Using OpenSCAD and **numpy-stl** I calculated the moments of inertia for the propeller. This non-dimensional number needs to be multiplied by the propeller mass:

In [47]:
np_I = np.array(inertia)
KE_I = np_I * weight / u.gravity * u.inches**2
KE_I.to_base_units()

Magnitude,[[7.120777092599685e-08 1.2086348509529597e-17 9.911976720526673e-22] [1.2086348509529597e-17 7.10812814808122e-08 -3.3942138291733913e-09] [9.911976720526673e-22 -3.3942138291733913e-09 1.9646819769693285e-09]]
Units,kilogram meter second2
